In [0]:
dbutils.widgets.text("p_data_date", "")
v_data_date = dbutils.widgets.get("p_data_date")

In [0]:
dbutils.secrets.list(scope="fomular1-scope")

In [0]:
formular1_key = dbutils.secrets.get(scope="fomular1-scope", key="formuladl1-key-account")

In [0]:
spark.conf.set("fs.azure.account.key.fomular1dl1.dfs.core.windows.net", formular1_key)

In [0]:
constructors_schema = "constructorId INT, constructorRef STRING, name STRING, nationality STRING, url STRING"

In [0]:
constructors_df = spark.read.schema(constructors_schema).json(f"abfss://raw@fomular1dl1.dfs.core.windows.net/{v_data_date}/constructors.json")

In [0]:
constructors_df.show()

In [0]:
from pyspark.sql.functions import current_timestamp, lit

constructors_drop_df = constructors_df.drop("url")

final_df = constructors_drop_df.withColumnRenamed("constructorId", "constructor_id") \
                                        .withColumnRenamed("constructorRef", "constructor_ref") \
                                        .withColumn("ingestion_date", current_timestamp())\
                                        .withColumn("data_date", lit(v_data_date))
display(final_df)   

In [0]:

final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.constructors")

In [0]:
%sql 
SELECT * FROM f1_processed.constructors